In [22]:
!wget https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py

--2024-07-23 08:06:30--  https://raw.githubusercontent.com/KPF-bigkinds/BIGKINDS-LAB/main/KPF-BERT-NER/label.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11245 (11K) [text/plain]
Saving to: ‘label.py’

label.py            100%[===================>]  10.98K  --.-KB/s    in 0s      

2024-07-23 08:06:30 (64.0 MB/s) - ‘label.py’ saved [11245/11245]



In [ ]:
!pip install kss # korean sentence splitter

In [3]:
from transformers import AutoTokenizer, BertForTokenClassification, logging
logging.set_verbosity_error()
import sys, os, torch
import numpy as np
import label # label.py를 업로드 한 후
import kss

print(torch.cuda.is_available()) # 리소스를 GPU 계열로 선택한 이후

True


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("KPF/KPF-bert-ner")
model = AutoModelForTokenClassification.from_pretrained("KPF/KPF-bert-ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/455M [00:00<?, ?B/s]

In [5]:
def ner_predict(text):
    text = text.replace('\n','')
    model.to("cuda")

    sents = kss.split_sentences(text)
    decoding_ner_sentence = ""
    word_list = list()
    pred_str = list()

    #text to model input
    for idx, sent in enumerate(sents):

        sent = sent.replace(" ", "-")
        test_tokenized = tokenizer(sent, return_tensors="pt")

        test_input_ids = test_tokenized["input_ids"].to("cuda")
        test_attention_mask = test_tokenized["attention_mask"].to("cuda")
        test_token_type_ids = test_tokenized["token_type_ids"].to("cuda")

        inputs = {
            "input_ids" : test_input_ids,
            "attention_mask" : test_attention_mask,
            "token_type_ids" : test_token_type_ids
        }

        if inputs['input_ids'].size()[1] > 512:
            cnt = int(inputs['input_ids'].size()[1])

            inp_np = inputs['input_ids'].cpu().numpy()
            att_np = inputs['attention_mask'].cpu().numpy()
            tok_np = inputs['token_type_ids'].cpu().numpy()

            for i in range(cnt):
                slice_inp = inp_np[0][(i*512):((i+1)*512)]
                slice_att = att_np[0][(i * 512):((i + 1) * 512)]
                slice_tok = tok_np[0][(i * 512):((i + 1) * 512)]

                slice_inp = slice_inp.reshape(1, len(slice_inp))
                slice_att = slice_att.reshape(1, len(slice_att))
                slice_tok = slice_tok.reshape(1, len(slice_tok))

                slice_inp = torch.tensor(slice_inp)
                slice_att = torch.tensor(slice_att)
                slice_tok = torch.tensor(slice_tok)

                slice_inp = torch.tensor(slice_inp).to("cuda")
                slice_att = torch.tensor(slice_att).to("cuda")
                slice_tok = torch.tensor(slice_tok).to("cuda")

                slice_inputs = {
                    "input_ids": slice_inp,
                    "attention_mask": slice_att,
                    "token_type_ids": slice_tok
                }

                # predict
                outputs = model(**slice_inputs)

                token_predictions = outputs[0].argmax(dim=2)
                token_prediction_list = token_predictions.squeeze(0).tolist()

                pred = [label.id2label[l] for l in token_prediction_list]

                pred_str = np.concatenate((pred_str, pred))
        else:
            #predict
            outputs = model(**inputs)

            token_predictions = outputs[0].argmax(dim=2)
            token_prediction_list = token_predictions.squeeze(0).tolist()

            pred_str = [label.id2label[l] for l in token_prediction_list]
        tt_tokenized = tokenizer(sent).encodings[0].tokens

        # decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False
        _word = ""
        # word_list = list()

        #model output to text
        for i, (token, pred) in enumerate(zip(tt_tokenized, pred_str)):
            if i == 0 or i == len(pred_str) - 1:
                continue
            token = token.replace('#', '').replace("-", " ")

            if token == "":
                continue

            if 'B-' in pred:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'
                    word_list.append({"word" : _word, "label" : prev_entity_tag, "desc" : "1"})
                    _word = ""

                if token[0] == ' ':
                    token = list(token)
                    token[0] = ' <'
                    token = ''.join(token)
                    decoding_ner_sentence += token
                    _word += token
                else:
                    decoding_ner_sentence += '<' + token
                    _word += token
                is_prev_entity = True
                prev_entity_tag = pred[2:]
                is_there_B_before_I = True

            elif 'I-' in pred:
                decoding_ner_sentence += token
                _word += token

                if is_there_B_before_I is True:
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token
                    is_prev_entity = False
                    is_there_B_before_I = False
                    word_list.append({"word" : _word, "label" : prev_entity_tag, "desc" : label.ner_code[prev_entity_tag]})
                    _word = ""
                else:
                    decoding_ner_sentence += token

    # print("OUTPUT")
    # print("sentence : ", decoding_ner_sentence)
    # print("result : ", word_list)
    return word_list

In [6]:
input_text = """
더불어민주당 이재명 대표가 이른바 '성남FC 후원금 의혹' 사건과 관련해 오는 10일 검찰에 출석해 조사를 받는다.
민주당 안호영 수석대변인은 6일 국회 브리핑을 통해 "이 대표가 10일 오전 10시 30분에 수원지검 성남지청에 출석하는 일정이 합의됐다"고 밝혔다.
안 수석대변인은 "검찰과 변호인단이 출석 날짜를 조율했고, 그 날짜가 적당하다고 판단한 것"이라고 설명했다.
공개적으로 출석하느냐는 질문에는 "이 대표는 당당히 출석해서 입장을 말씀하신다고 했다"며 "구체적으로 어떤 사람과 갈지, 어떻게 할지는 지켜봐야 한다"고 말했다.
앞서 검찰은 이 사건과 관련해 이 대표에게 지난해 12월 28일 소환을 통보했으나, 이 대표는 미리 잡아 둔 일정이 있다며 출석을 거부했다.
다만 이 대표는 "가능한 날짜와 조사 방식에 대해 변호인을 통해 협의해서 결정하겠다"며 조사에 응하겠다는 뜻을 밝혔고, 이후 검찰이 다시 요청한 10∼12일 중에서 출석 일자를 조율해 왔다.
성남FC 후원금 의혹 사건은 이 대표가 성남시장 재직 시절 성남FC 구단주로 있으면서 2016∼2018년 네이버·두산건설 등 기업들로부터 160억여원의 후원금을 유치하고, 이들 기업은 건축 인허가나 토지 용도 변경 등 편의를 받았다는 내용이다.
이 대표는 2018년 당시 바른미래당 등으로부터 이 의혹으로 고발당했다. 현재 제3자 뇌물공여 혐의를 받는 피의자 신분이다.
이 대표가 취임 이후 검찰의 소환조사에 응하는 것은 처음이다.
검찰은 앞서 지난 8월에도 대선 과정에서 허위 사실을 공표했다는 혐의로 이 대표에게 소환을 통보했으나, 당시 이 대표는 출석을 거부하고 서면 답변서만 제출한 바 있다.
"""

In [11]:
result = ner_predict(input_text)

In [9]:
for entity in result:
  print(entity)

{'word': '더불어민주당', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '이재명', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '대표', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '성남FC', 'label': 'LCP_CITY', 'desc': '도시'}
{'word': '오는 10일', 'label': 'DT_DAY', 'desc': '날짜 절기'}
{'word': '검찰', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '민주당', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '안호영', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '수석대변인', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '6일', 'label': 'DT_DAY', 'desc': '날짜 절기'}
{'word': '국회', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '이', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '대표', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '10일', 'label': 'DT_DAY', 'desc': '날짜 절기'}
{'word': '오전 10시 30분', 'label': 'TI_DURATION', 'desc': '기간'}
{'word': '수원지검', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '성남지청', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '안', 'label': 'PS_NAME', 'desc': '인물'}
{'word':

In [19]:
text = "국방부 법무관리관실이 지난해 8월 해병대 채모 상병 순직 사건을 재검토하던 국방부 조사본부에 임성근 전 해병대 1사단장을 혐의자에서 제외해야 한다는 의견을 제시한 것으로 드러났다. 26일 박정훈 전 해병대 수사단장대령 측이 중앙군사법원 재판에서 확보한 법무관리관실의 변사사건 의견 요청 회신 문건에 따르면 지난해 8월 14일 법무관리관실은 임성근 1사단장의 경우 수색작전 관련 안전통제 대책을 제대로 강구하지 않는 등의 과실이 있으나 사망과의 인과관계는 명확하지 않아 경찰에서 추가 검토가 필요하다고 밝혔다. 국방부가 경찰로 사건을 이첩할 때 임 전 사단장을 인지통보서에 혐의자로 적시하지 말아야 한다는 의견을 제시한 것이다. 이후 조사본부는 임 전 사단장을 혐의자에서 제외하고 대대장 2명에게만 혐의를 적시해 경찰로 사건을 넘겼다. 법무관리실의 이같은 의견은 같은 날 조사본부가 법무관리실에 제출한 재검토 중간결과 보고서를 반박하는 취지로 풀이된다. 조사본부는 이 보고서에서 임 전 사단장에 대해 장화를 신고 수색하게 한 혐의 작전전개를 재촉한 혐의 적색티 작업 지시 혐의 등이 있다고 적시한 바 있다. 고위공직자범죄수사처는 국방부 수뇌부가 조사본부에게 사실상 임 전 사단장을 제외하라는 가이드라인을 제시한 것으로 보고 수사하고 있다. 유재은 국방부 법무관리관은 당시 임 전 사단장 등 특정인을 제외하라는 지시를 한 적이 없다는 입장이다."

In [20]:
result = ner_predict(text)

In [21]:
for entity in result:
  print(entity)

{'word': '국방부', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '지난해', 'label': 'DT_OTHERS', 'desc': '기타'}
{'word': '8월해병대', 'label': 'OGG_MILITARY', 'desc': '군사'}
{'word': '국방부', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '임성근', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '1사단장', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '26일', 'label': 'DT_DAY', 'desc': '날짜 절기'}
{'word': '박정훈', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '해병대', 'label': 'OGG_MILITARY', 'desc': '군사'}
{'word': '중앙군사법원', 'label': 'OGG_LAW', 'desc': '법률'}
{'word': '지난해', 'label': 'DT_DURATION', 'desc': '기간'}
{'word': '8월 14일임성근', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '1사단장', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '경찰', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '국방부', 'label': 'OGG_POLITICS', 'desc': '정부 행정'}
{'word': '경찰', 'label': 'CV_OCCUPATION', 'desc': '직업'}
{'word': '사단장', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '임', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '사

In [18]:
text = "한동훈 국민의힘 당대표 후보는 팬덤에 기댄 정치를 하지 않겠다고 말했다. 한 후보는 5일 밤 라디오 신율의 뉴스 정면 승부에서 여권 정치인 중 유일하게 팬덤을 보유하고 있다는 말에 너무 고맙다면서도 다만 저희같이 나이 드는 남자들은 주제 파악을 잘해야 된다며 제가 대단히 매력 있고 제가 너무 좋아서 모이신 건 아니라는 걸 잘 알고 있다고 했다. 그러면서 그동안 제가 가지고 있었던 포지션 스탠스 사회에 대한 입장에 공감해 모인 정도라고 생각한다며 그렇기 때무에 제가 여기서 조금만 엇나가면 얼마든지 없어질 분들이라고 생각한다. 지금은 격려와 응원의 대상이라기보다는 비판과 감시의 대상인 것도 제가 안다고 말했다. 이어 저를 좋아해 주시는 분들도 너무 고맙지만 그걸 잊지 않아주셨으면 좋겠다. 단적으로 저도 그런 마음가짐을 갖고 있다고 했다. 한 후보는 누가 팬클럽 카페 등이 10배 늘어나는 계기가 본인이 거기 가입해서 글을 남기는 경우다라고 제안하더라. 이재명 대표 같은 경우 재명이네 마을에서 그렇게 했잖냐며 정치 시작하면서 비대위원장 할 때 그런 제안을 하신 분이 있었는데 전 그건 안 하려고 한다고 못 박았다. 그 이유에 대해선 그런 식이 되면 서로 간 좋은 영향도 있겠지만 안 좋은 영향도 있을 것 같기에 그런 차원에서 안 할 것이라며 이 자리를 빌려서 대단히 고맙다는 말씀을 드리지만 저는 기본적으로는 감시와 비판의 대상이라고 강조했다."
result = ner_predict(text)
for entity in result:
  print(entity)

{'word': '한동훈', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '국민의힘', 'label': 'OGG_OTHERS', 'desc': '기타'}
{'word': '당대표', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '5일', 'label': 'DT_DAY', 'desc': '날짜 절기'}
{'word': '밤', 'label': 'TI_DURATION', 'desc': '기간'}
{'word': '라디오', 'label': 'OGG_MEDIA', 'desc': '미디어 방송'}
{'word': '정치인', 'label': 'CV_OCCUPATION', 'desc': '직업'}
{'word': '10배', 'label': 'QT_PERCENTAGE', 'desc': '비율'}
{'word': '이재명', 'label': 'PS_NAME', 'desc': '인물'}
{'word': '대표', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
{'word': '비대위원장', 'label': 'CV_POSITION', 'desc': '직위, 직책'}
